# Assignment 4

## Description

In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with `pearsonr`, so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
%%capture
#RUN FIRST, installs a missing library
import sys
!{sys.executable} -m pip install lxml==4.4.1

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import ttest_rel
import re

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option("future.no_silent_downcasting", True)

In [4]:
nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

cities = cities.set_index('Metropolitan area')
ny_pop = cities.loc['New York City']['Population (2016 est.)[8]']
la_pop = cities.loc['Los Angeles']['Population (2016 est.)[8]']
cities = cities.reset_index()

diff_team_same_city = {'Metropolitan area':['New York City','New York City','Los Angeles'],
                       'Population (2016 est.)[8]':[ny_pop,ny_pop,la_pop],
                       'NHL':['Rangers','Islanders','Kings']}
df_diff_team = pd.DataFrame(diff_team_same_city)

cities = pd.concat([cities,df_diff_team])

cities['NHL'] = cities['NHL'].apply(lambda x: re.split(r"[\[\]]",x)[0])
cities['NHL last word'] = cities['NHL'].apply(lambda x: x.split(" ")[-1])


nhl_df['team'] = nhl_df['team'].apply(lambda x: x.replace("*",""))
nhl_df['team last word'] = nhl_df['team'].apply(lambda x: x.split(" ")[-1])
df = pd.merge(nhl_df,cities,left_on='team last word',right_on='NHL last word')

In [5]:
df

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,...,year,League,team last word,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL,NHL last word
0,Tampa Bay Lightning,82,54,23,5,113,.689,296,236,0.66,...,2018,NHL,Lightning,Tampa Bay Area,3032171,Buccaneers,Rays,—,Lightning,Lightning
1,Boston Bruins,82,50,20,12,112,.683,270,214,0.62,...,2018,NHL,Bruins,Boston,4794447,Patriots[note 14],Red Sox[note 15],Celtics,Bruins,Bruins
2,Toronto Maple Leafs,82,49,26,7,105,.640,277,232,0.49,...,2018,NHL,Leafs,Toronto,5928040,— [note 22],Blue Jays,Raptors[note 23],Maple Leafs,Leafs
3,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,...,2018,NHL,Panthers,Miami–Fort Lauderdale,6066387,Dolphins,Marlins,Heat,Panthers,Panthers
4,Detroit Red Wings,82,30,39,13,73,.445,217,255,-0.48,...,2018,NHL,Wings,Detroit,4297617,Lions,Tigers[note 20],Pistons[note 21],Red Wings,Wings
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,Los Angeles Kings,82,46,28,8,100,.610,206,174,0.40,...,2014,NHL,Kings,Los Angeles,13310447,NaN,NaN,NaN,Kings,Kings
147,Phoenix Coyotes,82,37,30,15,89,.543,216,231,-0.16,...,2014,NHL,Coyotes,Phoenix,4661537,Cardinals,Diamondbacks,Suns,Coyotes,Coyotes
148,Vancouver Canucks,82,36,35,11,83,.506,196,223,-0.30,...,2014,NHL,Canucks,Vancouver,2463431,—,—,[note 60],Canucks,Canucks
149,Calgary Flames,82,35,40,7,77,.470,209,241,-0.35,...,2014,NHL,Flames,Calgary,1392609,—,—,—,Flames,Flames


In [6]:
def nhl_correlation(): 
    # YOUR CODE HERE
    
    cols1 = ['team', 'W', 'L', 'NHL last word', 'year', 'Population (2016 est.)[8]', 'Metropolitan area']

    df1 = df[cols1]

    df1 = df1.set_index('year')
    df1 = df1.loc[2018]

    df1['W/L ratio'] = (df1['W'].astype('float'))/(df1['W'].astype('float')+df1['L'].astype('float'))
    df1['Population (2016 est.)[8]'] = df1['Population (2016 est.)[8]'].astype('float')

    df1=df1.groupby(['Metropolitan area']).mean(numeric_only =True)

    win_loss_by_region = df1['W/L ratio']
    population_by_region = df1['Population (2016 est.)[8]']

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"

    corr1, pval=stats.pearsonr(population_by_region, win_loss_by_region)
    return corr1

nhl_correlation()

0.012486162921209895

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [7]:
nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

cities = cities.set_index('Metropolitan area')
ny_pop = cities.loc['New York City']['Population (2016 est.)[8]']
la_pop = cities.loc['Los Angeles']['Population (2016 est.)[8]']
cities = cities.reset_index()

diff_nba_team_same_city = {'Metropolitan area':['New York City','Los Angeles'],
                       'Population (2016 est.)[8]':[ny_pop,la_pop],
                       'NBA':['Knicks','Lakers']}

df_diff_team2 = pd.DataFrame(diff_nba_team_same_city)
cities = pd.concat([cities,df_diff_team2])

pattern = r"[*]*[\xa0][\(\d\)]"

nba_df['team'] = nba_df['team'].apply(lambda x: re.split(pattern,x)[0])
nba_df['team'] = nba_df['team'].apply(lambda x: x.replace(r"[*]",""))
nba_df['team last word'] = nba_df['team'].apply(lambda x: x.split(" ")[-1])


cities['NBA'] = cities['NBA'].apply(lambda x: re.split(r"[\[\]]",x)[0])
cities['NBA last word'] = cities['NBA'].apply(lambda x: x.split(" ")[-1])

df2 = pd.merge(nba_df,cities,left_on='team last word',right_on='NBA last word')

cols2 = ['Metropolitan area','year','team','W/L%','Population (2016 est.)[8]','NBA last word']

df2 = df2[cols2]
df2 = df2.set_index('year')
df2 = df2.loc[2018]

df2['Population (2016 est.)[8]'] = df2['Population (2016 est.)[8]'].astype('float')
df2['W/L%'] = df2['W/L%'].astype('float')

df2 = df2.groupby(['Metropolitan area']).mean(numeric_only=True)

In [8]:
def nba_correlation():
    # YOUR CODE HERE
    #raise NotImplementedError()
    

    population_by_region = df2['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df2['W/L%'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    corr2, pval2=stats.pearsonr(population_by_region, win_loss_by_region)
    return corr2

nba_correlation()

-0.1763635064218294

## Question 3

For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [9]:
mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

cities = cities.set_index('Metropolitan area')
ny_pop = cities.loc['New York City']['Population (2016 est.)[8]']
la_pop = cities.loc['Los Angeles']['Population (2016 est.)[8]']
sf_pop = cities.loc['San Francisco Bay Area']['Population (2016 est.)[8]']
chi_pop = cities.loc['Chicago']['Population (2016 est.)[8]']
cities = cities.reset_index()

diff_mlb_team_same_city = {'Metropolitan area':['New York City','Los Angeles','San Francisco Bay Area','Chicago'],
                       'Population (2016 est.)[8]':[ny_pop,la_pop,sf_pop,chi_pop],
                       'MLB':['Yankees','Dodgers','Giants','Cubs']}

df_diff_team3 = pd.DataFrame(diff_mlb_team_same_city)
cities = pd.concat([cities,df_diff_team3])

mlb_df['team'] = mlb_df['team'].replace({'Boston Red Sox':'Boston RedSox',
                                       'Chicago White Sox':'Chicago WhiteSox'})
mlb_df['team last word'] = mlb_df['team'].apply(lambda x: x.split(" ")[-1])

cities['MLB'] = cities['MLB'].apply(lambda x: re.split(r"[\[\]]",x)[0])
cities['MLB'] = cities['MLB'].replace({'Red Sox':'RedSox',
                                       'Cubs White Sox':'WhiteSox'})
cities['MLB last word'] = cities['MLB'].apply(lambda x: x.split(" ")[-1])

df3 = pd.merge(mlb_df,cities,left_on='team last word',right_on='MLB last word')
cols3 = ['Metropolitan area','team','W-L%','year','Population (2016 est.)[8]']
df3 = df3[cols3]
df3 = df3.set_index('year')
df3 = df3.loc[2018]

df3['Population (2016 est.)[8]'] = df3['Population (2016 est.)[8]'].astype(float)
df3['W-L%'] = df3['W-L%'].astype(float)

df3 = df3.groupby(['Metropolitan area']).mean(numeric_only=True)

In [10]:
def mlb_correlation(): 
    # YOUR CODE HERE
    
    population_by_region = df3['Population (2016 est.)[8]']  # pass in metropolitan area population from cities
    win_loss_by_region = df3['W-L%'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"
    
    corr3, pval3=stats.pearsonr(population_by_region, win_loss_by_region)
    return corr3

mlb_correlation()

0.15003737475409498

## Question 4

For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data

In [11]:
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

cities = cities.set_index('Metropolitan area')
ny_pop = cities.loc['New York City']['Population (2016 est.)[8]']
la_pop = cities.loc['Los Angeles']['Population (2016 est.)[8]']
sf_pop = cities.loc['San Francisco Bay Area']['Population (2016 est.)[8]']
cities = cities.reset_index()

diff_nfl_team_same_city = {'Metropolitan area':['New York City','Los Angeles','San Francisco Bay Area'],
                       'Population (2016 est.)[8]':[ny_pop,la_pop,sf_pop],
                       'NFL':['Giants','Rams','49ers']}

df_diff_team4 = pd.DataFrame(diff_nfl_team_same_city)
cities = pd.concat([cities,df_diff_team4])

nfl_df['team'] = nfl_df['team'].apply(lambda x: x.replace('*',''))
nfl_df['team'] = nfl_df['team'].apply(lambda x: x.replace('+',''))
nfl_df['team last word'] = nfl_df['team'].apply(lambda x: x.split(" ")[-1])

cities['NFL'] = cities['NFL'].apply(lambda x: re.split(r"[\[\]]",x)[0])
cities['NFL last word'] = cities['NFL'].apply(lambda x: x.split(" ")[-1])

df4 = pd.merge(nfl_df,cities,left_on='team last word',right_on='NFL last word')
cols4 = ['Metropolitan area','team','W-L%','year','Population (2016 est.)[8]']

df4 = df4[cols4]
df4 = df4.set_index('year')
df4 = df4.loc[2018]

df4['Population (2016 est.)[8]'] = df4['Population (2016 est.)[8]'].astype(float)
df4['W-L%'] = df4['W-L%'].astype(float)

df4 = df4.groupby(['Metropolitan area']).mean(numeric_only=True)

In [12]:
def nfl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    population_by_region = df4['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df4['W-L%'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    corr4, pval4=stats.pearsonr(population_by_region, win_loss_by_region)

    return corr4
    
nfl_correlation()

0.004282141436393022